<div align="center"> 

---

# **Inmunofluorescence metrics**

</div>


---

<div align="center"> 

![Inmuno.png](attachment:2)

</div>

---
# <u> **Description:**</u>

Inmunofluorescence metrics

---

## **Workflow**

1. General preparation
2. Data preparation
3. Analysis
4. Exploratory data analysis
5. Visualization
6. Export data

---

**Variables to consider:** 
  - Session
  - Sex  
  
**ROIs:**
  - Prelimbic
  - Hippocampus (CA2, CA3)
  - Insula
  
**Analysis:**
  - 
  - 

---

### **General settings options**

In [1]:
addTaskCallback(function(...) {set.seed(42);TRUE})
options(warn=-1) # to turn off warnings
#options(warn=0) # to turn on warnings
options(repr.plot.width=14, repr.plot.height=10) # to resize plots

2 
2

In [2]:
setwd("../")
load("Data/Alcohol_consumption.RData")

In [3]:
load("Data/Behavior_metricsEPM.RData")

In [4]:
load("Data/Behavior_metricsNOR.RData")

In [5]:
load("Data/Behavior_metricsCPP.RData")

## **1. General preparation**

### **1.1 Load  R packages and dependencies.** 

  The following packages are needed for the to move on, firstly it is needed to download pacman library

 ---

In [6]:
# Make sure to install pacman before we begin
  if (!require("pacman")) {
  install.packages("pacman")
  }

# Load required packages  
  pacman::p_load(tidyverse,readxl,lme4,devtools,ggpubr,rstatix,ggdist,ggmap,data.table,cowplot,corrplot,magrittr,gghalves,skimr,emmeans,multcomp,easystats)


Loading required package: pacman



In [7]:
pal_batch <- c("#264653","#2a9d8f","#5e548e","#ff595e","#03045e")

## **2. Data preparation**

Reading files and set it all up.

In [8]:
Class_group <- read_csv("Data/Class.csv")

Rows: 63 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (3): RID, class, Sex

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
sheets <- excel_sheets("Data/Inmunos/Inmunos_EtOH.xlsx")

Inmuno_excel <- sheets %>% map(~ read_excel("Data/Inmunos/Inmunos_EtOH.xlsx",sheet = .x,na = c("NA","na")) %>% .[c(1:8,10:15)] %>% make_clean_names() %>% 
                               set_colnames(c("RID","Group","Sex","Intake","ROI","Hemisphere","Slice",
                                              "Filename","Iba_area","Iba_count","NeuN_area","NeuN_count","GFAP_area","GFAP_count"))) %>% 
                set_names(sheets) %>% reduce(rbind) %>% 
                mutate(Group = factor(Group, levels = c("Ctrl","Alc")),
                       Sex = factor(Sex, levels = c("female","male")),
                       Intake = case_when(Intake == "ctrl" ~ "Ctrl", 
                                          Intake == "low" ~ "Low", 
                                          Intake == "high" ~ "High"),
                       Intake = factor(Intake, levels = c("Ctrl","High","Low")),
                       ROI = factor(ROI),
                       RID = factor(RID),
                       Hemisphere = factor(Hemisphere),
                       Slice = factor(Slice))

New names:
• `` -> `...18`


### **2.1 Arranging**

In [10]:
m=Inmuno_excel %>% pivot_longer(cols = c(10,12,14), names_to = "Antibody", values_to = "Count") %>%
    mutate(Antibody = str_remove(Antibody,"_count")) %>% .[-c(9:11)]
n=Inmuno_excel %>% pivot_longer(cols = c(9,11,13), names_to = "Antibody", values_to = "Area") %>%
    mutate(Antibody = str_remove(Antibody,"_area")) %>% .[-c(9:11)]
Inmuno <- m %>% left_join(n) %>% mutate(Antibody = factor(Antibody))

Joining with `by = join_by(RID, Group, Sex, Intake, ROI, Hemisphere, Slice, Filename, Antibody)`


## **3. Analysis**

## **4. Exploratory data analysis**

In [11]:
Inmuno %>% skimr::skim()

,skim_type,skim_variable,n_missing,complete_rate,character.min,character.max,character.empty,character.n_unique,character.whitespace,factor.ordered,⋯,logical.mean,logical.count,numeric.mean,numeric.sd,numeric.p0,numeric.p25,numeric.p50,numeric.p75,numeric.p100,numeric.hist
,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<lgl>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,character,Filename,0,1.0000000,24,42,0,69,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,factor,RID,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,factor,Group,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,factor,Sex,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,factor,Intake,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,factor,ROI,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,factor,Hemisphere,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,factor,Slice,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,factor,Antibody,0,1.0000000,NA,NA,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             222       
Number of columns          11        
_______________________              
Column type frequency:               
  character                1         
  factor                   8         
  logical                  1         
  numeric                  1         
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 Filename              0             1  24  42     0       69          0

── Variable type: factor ───────────────────────────────────────────────────────
  skim_variable n_missing complete_rate ordered n_unique
1 RID                   0             1 FALSE         13
2 Group                 0             1 FALSE          2
3 Sex               

In [12]:
Inmuno %>% group_by(ROI,Antibody) %>% identify_outliers(Count) %>% filter(is.extreme==TRUE)

ROI,Antibody,RID,Group,Sex,Intake,Hemisphere,Slice,Filename,Count,Area,is.outlier,is.extreme
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>
prl,GFAP,R07,Alc,female,Low,both,C,R07_ALC_FEMALE_LOW_PRL_BOTH_C,1728,NA,TRUE,TRUE
prl,GFAP,R07,Alc,female,Low,both,D,R07_ALC_FEMALE_LOW_PRL_BOTH_D,1731,NA,TRUE,TRUE


### **4.1 Stats (hyphotesis testing)**

In [13]:
Inmuno.aov <- Inmuno %>% group_by(Antibody) %>% anova_test(dv = Count, 
                               between = c(Group,Sex,ROI))
Inmuno.aov

,Antibody,Effect,DFn,DFd,F,p,p<.05,ges
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,GFAP,Group,1,57,4.041,4.90e-02,*,6.60e-02
2,GFAP,Sex,1,57,6.929,1.10e-02,*,1.08e-01
3,GFAP,ROI,1,57,247.744,2.07e-22,*,8.13e-01
4,GFAP,Group:Sex,1,57,1.097,2.99e-01,,1.90e-02
5,GFAP,Group:ROI,1,57,1.090,3.01e-01,,1.90e-02
6,GFAP,Sex:ROI,1,57,7.177,1.00e-02,*,1.12e-01
7,GFAP,Group:Sex:ROI,1,57,2.227,1.41e-01,,3.80e-02
8,Iba,Group,1,62,4.265,4.30e-02,*,6.40e-02
9,Iba,Sex,1,62,11.703,1.00e-03,*,1.59e-01


In [14]:
Inmuno.t <- Inmuno %>% 
  group_by(Antibody,ROI) %>%
  pairwise_t_test(
    Count ~ Group, paired = FALSE, 
    p.adjust.method = "fdr")
Inmuno.t

,ROI,Antibody,.y.,group1,group2,n1,n2,p,p.signif,p.adj,p.adj.signif
,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,<chr>
1,CA2,GFAP,Count,Ctrl,Alc,22,24,1.27e-05,****,1.27e-05,****
2,prl,GFAP,Count,Ctrl,Alc,10,18,1.37e-01,ns,1.37e-01,ns
3,CA2,Iba,Count,Ctrl,Alc,22,24,3.40e-06,****,3.40e-06,****
4,prl,Iba,Count,Ctrl,Alc,10,18,5.11e-01,ns,5.11e-01,ns
5,CA2,NeuN,Count,Ctrl,Alc,22,24,6.04e-07,****,6.04e-07,****
6,prl,NeuN,Count,Ctrl,Alc,10,18,8.06e-02,ns,8.06e-02,ns


In [15]:
Inmuno.sex <- Inmuno %>%
  group_by(Antibody,ROI,Group) %>%
  pairwise_t_test(
    Count ~ Sex, paired = FALSE, 
    p.adjust.method = "fdr")
Inmuno.sex

,Group,ROI,Antibody,.y.,group1,group2,n1,n2,p,p.signif,p.adj,p.adj.signif
,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,<chr>
1,Ctrl,CA2,GFAP,Count,female,male,7,15,6.75e-02,ns,6.75e-02,ns
2,Alc,CA2,GFAP,Count,female,male,7,17,2.41e-01,ns,2.41e-01,ns
3,Ctrl,prl,GFAP,Count,female,male,5,5,1.44e-01,ns,1.44e-01,ns
4,Alc,prl,GFAP,Count,female,male,11,7,3.06e-02,*,3.06e-02,*
5,Ctrl,CA2,Iba,Count,female,male,7,15,1.30e-02,*,1.30e-02,*
6,Alc,CA2,Iba,Count,female,male,7,17,1.30e-01,ns,1.30e-01,ns
7,Ctrl,prl,Iba,Count,female,male,5,5,1.10e-01,ns,1.10e-01,ns
8,Alc,prl,Iba,Count,female,male,11,7,6.86e-02,ns,6.86e-02,ns
9,Ctrl,CA2,NeuN,Count,female,male,7,15,9.30e-02,ns,9.30e-02,ns


In [16]:
Inmuno.t_sex <- Inmuno %>%
  group_by(Antibody,ROI,Sex) %>%
  pairwise_t_test(
    Count ~ Group, paired = FALSE, 
    p.adjust.method = "fdr")
Inmuno.t_sex

,Sex,ROI,Antibody,.y.,group1,group2,n1,n2,p,p.signif,p.adj,p.adj.signif
,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,<chr>
1,female,CA2,GFAP,Count,Ctrl,Alc,7,7,8.24e-02,ns,8.24e-02,ns
2,male,CA2,GFAP,Count,Ctrl,Alc,15,17,5.66e-05,****,5.66e-05,****
3,female,prl,GFAP,Count,Ctrl,Alc,5,11,1.59e-01,ns,1.59e-01,ns
4,male,prl,GFAP,Count,Ctrl,Alc,5,7,8.11e-01,ns,8.11e-01,ns
5,female,CA2,Iba,Count,Ctrl,Alc,7,7,5.74e-03,**,5.74e-03,**
6,male,CA2,Iba,Count,Ctrl,Alc,15,17,2.30e-04,***,2.30e-04,***
7,female,prl,Iba,Count,Ctrl,Alc,5,11,4.81e-01,ns,4.81e-01,ns
8,male,prl,Iba,Count,Ctrl,Alc,5,7,5.11e-01,ns,5.11e-01,ns
9,female,CA2,NeuN,Count,Ctrl,Alc,7,7,1.45e-02,*,1.45e-02,*


In [17]:
Inmuno.aov_class <- Inmuno %>% group_by(Antibody) %>% anova_test(dv = Count,
                               between = c(Intake,Sex,ROI))
Inmuno.aov_class

,Antibody,Effect,DFn,DFd,F,p,p<.05,ges
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,GFAP,Intake,2,53,8.876,4.73e-04,*,0.251
2,GFAP,Sex,1,53,7.049,1.00e-02,*,0.117
3,GFAP,ROI,1,53,310.293,8.26e-24,*,0.854
4,GFAP,Intake:Sex,2,53,0.664,5.19e-01,,0.024
5,GFAP,Intake:ROI,2,53,5.568,6.00e-03,*,0.174
6,GFAP,Sex:ROI,1,53,8.132,6.00e-03,*,0.133
7,GFAP,Intake:Sex:ROI,2,53,2.571,8.60e-02,,0.088
8,Iba,Intake,2,58,6.448,3.00e-03,*,0.182
9,Iba,Sex,1,58,17.312,1.06e-04,*,0.230


In [18]:
Inmuno.t_class <- Inmuno %>%
  group_by(Antibody,ROI,Sex) %>%
  pairwise_t_test(
    Count ~ Intake, 
    p.adjust.method = "fdr")
Inmuno.t_class

,Sex,ROI,Antibody,.y.,group1,group2,n1,n2,p,p.signif,p.adj,p.adj.signif
,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,<chr>
1,female,CA2,GFAP,Count,Ctrl,High,7,3,3.23e-02,*,9.68e-02,ns
2,female,CA2,GFAP,Count,Ctrl,Low,7,4,3.71e-01,ns,3.71e-01,ns
3,female,CA2,GFAP,Count,High,Low,3,4,1.75e-01,ns,2.63e-01,ns
4,male,CA2,GFAP,Count,Ctrl,High,15,10,1.78e-02,*,1.78e-02,*
5,male,CA2,GFAP,Count,Ctrl,Low,15,7,1.98e-08,****,5.94e-08,****
6,male,CA2,GFAP,Count,High,Low,10,7,5.89e-05,****,8.83e-05,****
7,female,prl,GFAP,Count,Ctrl,High,5,3,5.80e-01,ns,5.80e-01,ns
8,female,prl,GFAP,Count,Ctrl,Low,5,8,2.01e-02,*,3.01e-02,*
9,female,prl,GFAP,Count,High,Low,3,8,1.38e-02,*,3.01e-02,*


In [19]:
Inmuno.t_class_sex <- Inmuno %>%
  group_by(Antibody,ROI,Intake) %>%
  pairwise_t_test(
    Count ~ Sex, 
    p.adjust.method = "fdr")
Inmuno.t_class_sex

,Intake,ROI,Antibody,.y.,group1,group2,n1,n2,p,p.signif,p.adj,p.adj.signif
,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,<chr>
1,Ctrl,CA2,GFAP,Count,female,male,7,15,0.06750,ns,0.06750,ns
2,High,CA2,GFAP,Count,female,male,3,10,0.12200,ns,0.12200,ns
3,Low,CA2,GFAP,Count,female,male,4,7,0.00543,**,0.00543,**
4,Ctrl,prl,GFAP,Count,female,male,5,5,0.14400,ns,0.14400,ns
5,High,prl,GFAP,Count,female,male,3,3,0.16500,ns,0.16500,ns
6,Low,prl,GFAP,Count,female,male,8,4,0.05510,ns,0.05510,ns
7,Ctrl,CA2,Iba,Count,female,male,7,15,0.01300,*,0.01300,*
8,High,CA2,Iba,Count,female,male,3,10,0.97300,ns,0.97300,ns
9,Low,CA2,Iba,Count,female,male,4,7,0.05220,ns,0.05220,ns


### **4.1 Stats (Linear modelling)**

In [20]:
x <- Inmuno$Antibody %>% levels()
y <- Inmuno$ROI %>% levels()
Model_AI <- Inmuno %>% split(y) %>% 
            map(~ .x %>% split(x) %>% map(~ .x %>% lm(formula = Count ~ Group*Sex)) )

Model_AI_Intake <- Inmuno %>% split(y) %>% 
            map(~ .x %>% split(x) %>% map(~ .x %>% lm(formula = Count ~ Intake*Sex)) )

In [ ]:
report(Model_AI_Intake$CA2$GFAP)
report(Model_AI_Intake$prl$Iba)

In [21]:
Model_AI_Intake %>% map(~ .x %>% summary())

$CA2
     Length Class Mode
GFAP 14     lm    list
Iba  14     lm    list
NeuN 14     lm    list

$prl
     Length Class Mode
GFAP 14     lm    list
Iba  14     lm    list
NeuN 14     lm    list


In [51]:
emmeans_AI <- Model_AI$CA2 %>% map(~ .x %>% emmeans(specs = pairwise ~ Group|Sex, adjust="none"))
emmeans_AI %>% map(~ .x$contrasts %>% as_tibble() %>% filter(p.value < 0.05) %>% add_significance())

contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


In [49]:
emmeans_AI <- Model_AI$prl %>% map(~ .x %>% emmeans(specs = pairwise ~ Group|Sex, adjust="none"))
emmeans_AI %>% map(~ .x$contrasts %>% as_tibble() %>% filter(p.value < 0.05) %>% add_significance())

contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


In [48]:
emmeans_AI_Intake <- Model_AI_Intake$CA2 %>% map(~ .x %>% 
                                                 emmeans(specs = pairwise ~ Intake|Sex, adjust="none")) 
emmeans_AI_Intake %>% map(~ .x$contrasts %>% as_tibble() %>% filter(p.value < 0.05) %>% add_significance())

contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


In [47]:
emmeans_AI_Intake <- Model_AI_Intake$prl %>% map(~ .x %>% 
                                                 emmeans(specs = pairwise ~ Intake|Sex, adjust="none"))
emmeans_AI_Intake %>% map(~ .x$contrasts %>% as_tibble() %>% filter(p.value < 0.05) %>% add_significance())

contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
contrast,Sex,estimate,SE,df,t.ratio,p.value,p.value.signif
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


## **5. Visualization**


In [63]:
Sex.Plot_Inmuno.AI_male <- Inmuno$ROI %>% levels() %>% 
    map(~ Inmuno %>% filter(ROI == .x, Sex == "male") %>%
  ggplot(aes(x = Group,y = Count, 
             fill = Group)) + facet_grid(. ~ Antibody) +
  stat_halfeye(
      adjust = .8, width = .7, fill = "grey85",
      interval_colour = NA, 
      position = position_nudge(x = .01),
      aes(thickness = stat(f*n))) +
  xlab("Group") + ylab("Cell count") + 
  scale_color_manual(values = pal_group) +
  scale_fill_manual(values = pal_group) +
  geom_half_point(side = "l", aes(color = Group, fill = Group),
                  range_scale = .1, alpha = .4, size = 3) +
  geom_boxplot(width = .2) +
  theme_pubr() + theme(legend.position = "none") +
  stat_compare_means(aes(label = after_stat(p.signif)), 
                  method = "t.test", ref.group = "Ctrl", hide.ns = TRUE) + 
  theme_settings) %>% set_names(Inmuno$ROI %>% levels())

In [64]:
Sex.Plot_Inmuno.AI_female <- Inmuno$ROI %>% levels() %>% 
    map(~ Inmuno %>% filter(ROI == .x, Sex == "female") %>%
  ggplot(aes(x = Group,y = Count, 
             fill = Group)) + facet_grid(. ~ Antibody) +
  stat_halfeye(
      adjust = .8, width = .7, fill = "grey85",
      interval_colour = NA, 
      position = position_nudge(x = .01),
      aes(thickness = stat(f*n))) +
  xlab("Group") + ylab("Cell count") + 
  scale_color_manual(values = pal_group) +
  scale_fill_manual(values = pal_group) +
  geom_half_point(side = "l", aes(color = Group, fill = Group),
                  range_scale = .1, alpha = .4, size = 3) +
  geom_boxplot(width = .2) +
  theme_pubr() + theme(legend.position = "none") +
  stat_compare_means(aes(label = after_stat(p.signif)), 
                  method = "t.test", ref.group = "Ctrl", hide.ns = TRUE) + 
  theme_settings) %>% set_names(Inmuno$ROI %>% levels())

In [65]:
Plot_Inmuno.AI_class <- Inmuno$ROI %>% levels() %>% 
    map(~ Inmuno %>% filter(ROI == .x) %>%
  ggplot(aes(x = Intake,y = Count, 
             fill = Intake)) + facet_grid(. ~ Antibody) +
  stat_halfeye(
      adjust = .8, width = .7, fill = "grey85",
      interval_colour = NA, 
      position = position_nudge(x = .01),
      aes(thickness = stat(f*n))) +
  xlab("Group") + ylab("Cellular count") + 
  scale_color_manual(values = pal_group) +
  scale_fill_manual(values = pal_group) +
  geom_half_point(side = "l", aes(color = Intake, fill = Intake),
                  range_scale = .1, alpha = .4, size = 3) +
  geom_boxplot(width = .2) +
  theme_pubr() + theme(legend.position = "none") +
  stat_compare_means(aes(label = after_stat(p.signif)), 
                  method = "t.test", ref.group = "Ctrl", hide.ns = TRUE) + 
  theme_settings) %>% set_names(Inmuno$ROI %>% levels())

In [66]:
Sex.Plot_Inmuno.AI_class_male <- Inmuno$ROI %>% levels() %>% 
    map(~ Inmuno %>% filter(ROI == .x, Sex == "male") %>%
  ggplot(aes(x = Intake,y = Count, 
             fill = Intake)) + facet_grid(. ~ Antibody) +
  stat_halfeye(
      adjust = .8, width = .7, fill = "grey85",
      interval_colour = NA, 
      position = position_nudge(x = .01),
      aes(thickness = stat(f*n))) +
  xlab("Group") + ylab("Cell count") + 
  scale_color_manual(values = pal_group) +
  scale_fill_manual(values = pal_group) +
  geom_half_point(side = "l", aes(color = Intake, fill = Intake),
                  range_scale = .1, alpha = .4, size = 3) +
  geom_boxplot(width = .2) +
  theme_pubr() + theme(legend.position = "none") +
  stat_compare_means(aes(label = after_stat(p.signif)), 
                  method = "t.test", ref.group = "Ctrl", hide.ns = TRUE) + 
  theme_settings) %>% set_names(Inmuno$ROI %>% levels())

In [67]:
Sex.Plot_Inmuno.AI_class_female <- Inmuno$ROI %>% levels() %>% 
    map(~ Inmuno %>% filter(ROI == .x, Sex == "female") %>%
  ggplot(aes(x = Intake,y = Count, 
             fill = Intake)) + facet_grid(. ~ Antibody) +
  stat_halfeye(
      adjust = .8, width = .7, fill = "grey85",
      interval_colour = NA, 
      position = position_nudge(x = .01),
      aes(thickness = stat(f*n))) +
  xlab("Group") + ylab("Cell count") + 
  scale_color_manual(values = pal_group) +
  scale_fill_manual(values = pal_group) +
  geom_half_point(side = "l", aes(color = Intake, fill = Intake),
                  range_scale = .1, alpha = .4, size = 3) +
  geom_boxplot(width = .2) +
  theme_pubr() + theme(legend.position = "none") +
  stat_compare_means(aes(label = after_stat(p.signif)), 
                  method = "t.test", ref.group = "Ctrl", hide.ns = TRUE) + 
  theme_settings) %>% set_names(Inmuno$ROI %>% levels())



## **6. Export data**

In [ ]:
dir.create("Figures/EPM")

In [ ]:
ggsave(plot = Plot_EPM.AI, dpi=300,height = 8, width = 12,bg="white",
       filename = "Figures/EPM/Plot_EPM_AI.png")

ggsave(plot = Plot_EPM.AI_class, dpi=300,height = 8, width = 12,bg="white",
       filename = "Figures/EPM/Plot_EPM_AI_class.png")

ggsave(plot = Sex.Plot_EPM.AI, dpi=300,height = 8, width = 12,bg="white",
       filename = "Figures/EPM/Sex_Plot_EPM_AI.png")

ggsave(plot = Sex.Plot_EPM.AI_class, dpi=300,height = 8, width = 12,bg="white",
       filename = "Figures/EPM/Sex_Plot_EPM_AI_class.png")

In [ ]:
save(Report_DLC,EPM_AI,Plot_EPM.AI,Plot_EPM.AI_class,Sex.Plot_EPM.AI,Sex.Plot_EPM.AI_class, file = paste0(getwd(),"/Data/Behavior_metricsEPM.RData"))